In [1]:
%matplotlib inline
import sys
sys.path.insert(0, '/home/cnaik/s/3/ilp')

import numpy as np
import operator
from  utils import entailment
import json
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
process =  'evaporation'

In [3]:
def load_srl_data(srl_file):
    d = json.load(open(srl_file, "r"))
    data = {}
    for p_data in d:
        process = p_data['process']
        ss_data = p_data['sentences']
        sent_to_id = {}
        id_to_args = {}
        arg_role_scores = {}
        arg_role_srl_data = {}
        for s_data in ss_data:
            sentence = s_data['text']
            s_id = s_data['sentenceId']
            sent_to_id[sentence] = s_id
            a_spans = s_data['predictionArgumentSpan']
            args = []
            for a_span in a_spans:
                srl_role_prediction = a_span['rolePredicted']
                start_idx = a_span['startIdx']
                end_idx = a_span['endIdx']
                arg_text = a_span['text']
                arg_id = a_span['argId']
                role_prob_list = a_span['probRoles']
                args.append((arg_id, arg_text))
                role_probs = {}
                for role_prob in role_prob_list:
                    role_probs.update(role_prob)
                arg_role_scores[(s_id, arg_id)] = role_probs
                arg_role_srl_data[(s_id, arg_id)] = [srl_role_prediction, start_idx, end_idx]
            id_to_args[s_id] = args
        data[process] = [sent_to_id, id_to_args, arg_role_scores, arg_role_srl_data]
    return data

def get_sentences(p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    return [(v, k) for k, v in sent_to_id.iteritems()]

def get_sentence_args(sentence, p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    s_id = sent_to_id[sentence]
    return id_to_args[s_id]

def get_role_scores(sentence, arg_id, role, p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    s_id = sent_to_id[sentence]
    return arg_role_scores[s_id, arg_id][role]

def get_similarity_score(arg1, arg2):
    ret = entailment.get_ai2_textual_entailment(arg1, arg2)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score1 = mean_a_score * confidence

    ret = entailment.get_ai2_textual_entailment(arg2, arg1)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score2 = mean_a_score * confidence
    return max(score1, score2)

In [4]:
srl_file = "/home/cnaik/s/3/ilp/data/test.srlpredict.json"
def load_srl_data(srl_file):
    d = json.load(open(srl_file, "r"))
    data = {}
    for p_data in d:
        process = p_data['process']
        ss_data = p_data['sentences']
        sent_to_id = {}
        id_to_args = {}
        arg_role_scores = {}
        arg_role_srl_data = {}
        for s_data in ss_data:
            sentence = s_data['text']
            s_id = s_data['sentenceId']
            sent_to_id[sentence] = s_id
            a_spans = s_data['predictionArgumentSpan']
            args = []
            for a_span in a_spans:
                srl_role_prediction = a_span['rolePredicted']
                start_idx = a_span['startIdx']
                end_idx = a_span['endIdx']
                arg_text = a_span['text']
                arg_id = a_span['argId']
                role_prob_list = a_span['probRoles']
                args.append((arg_id, arg_text))
                role_probs = {}
                for role_prob in role_prob_list:
                    role_probs.update(role_prob)
                arg_role_scores[(s_id, arg_id)] = role_probs
                arg_role_srl_data[(s_id, arg_id)] = [srl_role_prediction, start_idx, end_idx]
            id_to_args[s_id] = args
        data[process] = [sent_to_id, id_to_args, arg_role_scores, arg_role_srl_data]
    return data

In [5]:
def get_arg(output_map, context):
    for s, val in output_map.iteritems():
        args = id_to_args[s]
        for a, aval in val.iteritems():
            arg = dict(args)[a]
            if context == (s, a):
                return arg
    return None

In [6]:
def get_similarity_score(arg1, arg2):
    ret = entailment.get_ai2_textual_entailment(arg1, arg2)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score1 = mean_a_score * confidence

    ret = entailment.get_ai2_textual_entailment(arg2, arg1)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score2 = mean_a_score * confidence
    return max(score1, score2)


In [7]:
srl_data = load_srl_data()

TypeError: load_srl_data() takes exactly 1 argument (0 given)

In [8]:
sent_to_id, id_to_args, arg_role_scores, arg_role_srl_data = srl_data[process]

NameError: name 'srl_data' is not defined

In [9]:
id_to_sent = {v:k for k, v in sent_to_id.iteritems()}

NameError: name 'sent_to_id' is not defined

In [10]:
sentences = {v: k for k, v in sent_to_id.iteritems()}

NameError: name 'sent_to_id' is not defined

In [11]:
roles = ['undergoer', 'enabler', 'trigger', 'result', 'NONE']

f = open("../output/"+process+"_ilp.sol")
lines = f.readlines()
data = filter(lambda x: x.startswith('Z'), lines)
data =  map(lambda x: x[:-1], data)

In [12]:
output_map = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for d in data:
    var, ind = d.split(" ")
    var_ids = var.split("_")
    s = int(var_ids[1])
    a = int(var_ids[2])
    r = int(var_ids[3])
    output_map[s][a][r] = int(ind)

In [27]:
def get_sentences(p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    return [(v, k) for k, v in sent_to_id.iteritems()]

In [28]:
def get_sentence_args(sentence, p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    s_id = sent_to_id[sentence]
    return id_to_args[s_id]

In [30]:
def get_role_scores(sentence, arg_id, role, p_data):
    sent_to_id, id_to_args, arg_role_scores = p_data
    s_id = sent_to_id[sentence]
    return arg_role_scores[s_id, arg_id][role]

In [33]:
def get_similarity_score(arg1, arg2):
    ret = entailment.get_ai2_textual_entailment(arg1, arg2)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score1 = mean_a_score * confidence

    ret = entailment.get_ai2_textual_entailment(arg2, arg1)
    a_scores = map(lambda x: x['score'], ret['alignments'])
    if len(a_scores):
        mean_a_score = np.mean(a_scores)
    else:
        mean_a_score = 0

    confidence = ret['confidence'] if ret['confidence'] else 0
    score2 = mean_a_score * confidence
    return max(score1, score2)

In [31]:
# Integer Linear Programming for Joint Inference.
p_data = srl_data[process][:3]

sentences = get_sentences(p_data)

lambda_1 = 0.9
lambda_2 = 1 - lambda_1

role_score_vars = {}

for s_id, sentence in sentences:
    args = get_sentence_args(sentence, p_data)
    for a_id, arg in args:
        for r_id, role in enumerate(roles[:4]):
            role_score = get_role_scores(sentence, a_id, role, p_data)
            # data: role classifier scores
            role_score_vars[s_id, a_id, r_id] = role_score            

In [32]:
role_score_vars

{(21, 0, 0): 0.1807751121371668,
 (21, 0, 1): 0.15015688404441685,
 (21, 0, 2): 0.003585451703218445,
 (21, 0, 3): 0.040318862092067205,
 (23, 0, 0): 0.15786094076412716,
 (23, 0, 1): 0.05161655673880439,
 (23, 0, 2): 0.029092096361856612,
 (23, 0, 3): 0.04165401436523114,
 (41, 0, 0): 0.005450130390077135,
 (41, 0, 1): 0.005367232762490959,
 (41, 0, 2): 0.053657507799292126,
 (41, 0, 3): 0.9227275271664176,
 (62, 0, 0): 0.03278797713317905,
 (62, 0, 1): 0.7794224685978454,
 (62, 0, 2): 0.01241042533108172,
 (62, 0, 3): 0.010792789673365591,
 (62, 1, 0): 0.0010399857532388257,
 (62, 1, 1): 0.0035502172477538396,
 (62, 1, 2): 0.05011514332346223,
 (62, 1, 3): 0.004384997973040028,
 (62, 2, 0): 0.023559631911904355,
 (62, 2, 1): 0.0190756637261007,
 (62, 2, 2): 0.02392717460719851,
 (62, 2, 3): 0.0014319718133875082,
 (62, 3, 0): 0.15191941099531447,
 (62, 3, 1): 0.37218367636209915,
 (62, 3, 2): 0.009465212492560493,
 (62, 3, 3): 0.021345326391745554,
 (97, 0, 0): 0.013489166874428249,


In [ ]:
# # DEBUG

# ilp_score = {}
# for s_1, val_1 in output_map.iteritems():
#     args_1 = id_to_args[s_1]
#     for a_1, aval_1 in val_1.iteritems():
#         arg_1 = dict(args_1)[a_1]
#         for r, rv_1 in aval_1.iteritems():
# #             print s_1, a_1, r
#             # iterate through all other sentences
#             tmp = 0
#             for s_2, val_2 in output_map.iteritems():
#                 if s_1 != s_2:
#                     args_2 = id_to_args[s_2]
#                     for a_2, aval_2 in val_2.iteritems():
#                         arg_2 = dict(args_2)[a_2]
#                         rv_2 = aval_2[r]
#                         tmp += rv_2 * float(get_similarity_score(arg_1, arg_2))
# #                         print '\t', rv_2, float(get_similarity_score(arg_1, arg_2))
# #                         print '\t\t', arg_1, arg_2
#             #
# #             print tmp, lambda_2 * tmp
#             ilp_score[s_1, a_1, r] = (float(role_score_vars[s_1, a_1, r]) * lambda_1) + (lambda_2 * tmp)
# #             ilp_score[s_1, a_1, r] = rv_1 * ((float(role_score_vars[s_1, a_1, r]) * lambda_1) + (lambda_2 * tmp))
                    

In [51]:
ilp_score = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
for s_1, val_1 in output_map.iteritems():
    args_1 = id_to_args[s_1]
    for a_1, aval_1 in val_1.iteritems():
        arg_1 = dict(args_1)[a_1]
        for r, rv_1 in aval_1.iteritems():
            # iterate through all other sentences
            tmp = 0
            for s_2, val_2 in output_map.iteritems():
                if s_1 != s_2:
                    args_2 = id_to_args[s_2]
                    for a_2, aval_2 in val_2.iteritems():
                        arg_2 = dict(args_2)[a_2]
                        rv_2 = aval_2[r]
                        tmp += rv_2 * float(get_similarity_score(arg_1, arg_2))
            #
            ilp_score[s_1][a_1][r] = (float(role_score_vars[s_1, a_1, r]) * lambda_1) + (lambda_2 * tmp)

In [58]:
norm_ilp_score = defaultdict(lambda: defaultdict(int))
for s_id, s_data in ilp_score.iteritems():
    for a_id, a_data in s_data.iteritems():
        denom = sum(a_data.values())
        norm_vals = dict(map(lambda x: (x[0], x[1]/denom), a_data.items()))
        norm_ilp_score[s_id][a_id]= norm_vals
#         print dict(a_data)
#         print denom
#         print norm_vals
#         print sum(norm_vals.values())
        
        

In [59]:
norm_ilp_score

defaultdict(<function __main__.<lambda>>,
            {21: defaultdict(int,
                         {0: {0: 0.6042989171378661,
                           1: 0.27519290597222257,
                           2: 0.005739294584187134,
                           3: 0.11476888230572425}}),
             23: defaultdict(int,
                         {0: {0: 0.532756681841807,
                           1: 0.20459950770847152,
                           2: 0.1384072895725638,
                           3: 0.1242365208771576}}),
             41: defaultdict(int,
                         {0: {0: 0.22347942850934127,
                           1: 0.018544032468316905,
                           2: 0.04286964365328938,
                           3: 0.7151068953690525}}),
             62: defaultdict(int,
                         {0: {0: 0.09379823106013217,
                           1: 0.8728850131822985,
                           2: 0.013775650664518664,
                           3: 0.01954110

In [44]:
ilp_score

{(21, 0, 0): 0.33997801480561685,
 (21, 0, 1): 0.15482327571287063,
 (21, 0, 2): 0.0032289218527118612,
 (21, 0, 3): 0.06456886759381328,
 (23, 0, 0): 0.1863207155266962,
 (23, 0, 1): 0.07155447875540982,
 (23, 0, 2): 0.04840510894789317,
 (23, 0, 3): 0.043449173428204306,
 (41, 0, 0): 0.26430753166179854,
 (41, 0, 1): 0.021931895393907448,
 (41, 0, 2): 0.05070162284197977,
 (41, 0, 3): 0.8457518423510104,
 (62, 0, 0): 0.07614868538983796,
 (62, 0, 1): 0.7086385905050964,
 (62, 0, 2): 0.011183555133574455,
 (62, 0, 3): 0.015864152735956386,
 (62, 1, 0): 0.018424649309434212,
 (62, 1, 1): 0.0031989337820758092,
 (62, 1, 2): 0.145103947868667,
 (62, 1, 3): 0.005048125301920489,
 (62, 2, 0): 0.02120366872071392,
 (62, 2, 1): 0.01716809735349063,
 (62, 2, 2): 0.02153445714647866,
 (62, 2, 3): 0.015351274632048755,
 (62, 3, 0): 0.18618451611013667,
 (62, 3, 1): 0.3421236774929248,
 (62, 3, 2): 0.00853286357890535,
 (62, 3, 3): 0.0476418572398329,
 (97, 0, 0): 0.31352486991931844,
 (97, 0, 1

In [38]:
for s_id, args in id_to_args.items():
    print s_id, len(args)

577 3
229 1
97 1
470 1
393 1
631 2
621 1
272 1
340 1
21 1
214 1
41 1
23 1
505 9
62 4


In [39]:
id_to_args

{21: [(0, u'number joining the liquid state')],
 23: [(0, u'the pressure in a liquid-filled container')],
 41: [(0, u'make the water form clouds from water vapour and condensation')],
 62: [(2, u'evaporation'),
  (1, u'caused'),
  (0, u'the impingement of a second sound thermal pulse'),
  (3,
   u'the impingement of a second sound thermal pulse onto a he iivapor interface')],
 97: [(0, u'the conversion of water to gas')],
 214: [(0, u'salt from seawater')],
 229: [(0, u'water')],
 272: [(0,
   u'the water to cool rapidly to its freezing point and it solidifies suddenly .')],
 340: [(0, u'adequate air movement over the lagoon')],
 393: [(0, u'difference')],
 470: [(0, u'primary')],
 505: [(3, u'paired'),
  (5, u'your water level'),
  (1, u'check the water level - splashing around , paired with evaporation'),
  (0, u'causes'),
  (6, u'time'),
  (4, u'evaporation'),
  (7, u'over'),
  (2, u'decrease'),
  (8, u'your water level to decrease over time')],
 577: [(2, u'turns'), (1, u'liquid'),